In [12]:
import pandas as pd

# Load the CSV files
df60 = pd.read_csv("MSJO 60 01-03 2023.csv", delimiter=";")
df900 = pd.read_csv("MSJO 900 01-03 2023.csv", delimiter=";")

# Set PVLAUE to float
df60["PVALUE"] = df60["PVALUE"].str.replace(",", ".").astype(float)
df900["PVALUE"] = df900["PVALUE"].str.replace(",", ".").astype(float)

# Define a function to filter outliers for each MESS_ID
def filter_outliers(df):
    # Group-wise calculation of quartiles and upper thresholds
    grouped = df.groupby("MESS_ID")
    filtered_dfs = []

    for name, group in grouped:
        # Calculate quartiles
        Q1 = group["PVALUE"].quantile(0.25)
        Q3 = group["PVALUE"].quantile(0.75)
        IQR = Q3 - Q1
        
        # Calculate upper threshold for outliers
        upper_threshold = Q3 + 1.5 * IQR
        
        # Filter outliers
        filtered_group = group[(group["PVALUE"] >= Q1 - 1.5 * IQR) & (group["PVALUE"] <= Q3 + 1.5 * IQR)]
        
        # Append filtered group to list
        filtered_dfs.append(filtered_group)
    
    # Concatenate filtered dataframes
    df_filtered = pd.concat(filtered_dfs)
    
    return df_filtered

# Filter outliers for df60 and df900 separately
df60_filtered = filter_outliers(df60)
df900_filtered = filter_outliers(df900)

# Sort the filtered dataframes
df60_filtered.sort_values(by=["MESS_ID", "VALUEDATE"], ascending=[True, True], inplace=True)
df900_filtered.sort_values(by=["MESS_ID", "VALUEDATE"], ascending=[True, True], inplace=True)

# Save to CSV
df60_filtered.to_csv("cleaned60.csv", index=False)
df900_filtered.to_csv("cleaned900.csv", index=False)


In [ ]:
# ARIMA Modell
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

df60 = pd.read_csv("cleaned60.csv")
df900 = pd.read_csv("cleaned900.csv")

# Convert 'VALUEDATE' column to datetime and set it as index
df60['VALUEDATE'] = pd.to_datetime(df60['VALUEDATE'])
df60.set_index('VALUEDATE', inplace=True)

df900['VALUEDATE'] = pd.to_datetime(df900['VALUEDATE'])
df900.set_index('VALUEDATE', inplace=True)

# Calculate the average value for each DataFrame
df60['avg_value'] = df60.mean(axis=1)
df900['avg_value'] = df900.mean(axis=1)

# Resample 60-second data to 15-minute intervals and handle missing values
df_60s_resampled = df60['avg_value'].resample('15T').mean()
df_60s_resampled = df_60s_resampled.fillna(df_60s_resampled.mean())

# Ensure df900's avg_value is handled similarly
df900['avg_value'] = df900['avg_value'].fillna(df900['avg_value'].mean())

# Train-Test Split
train_size_60s = int(len(df_60s_resampled) * 0.8)
train_size_15min = int(len(df900) * 0.8)

train_60s, test_60s = df_60s_resampled[:train_size_60s], df_60s_resampled[train_size_60s:]
train_15min, test_15min = df900['avg_value'][:train_size_15min], df900['avg_value'][train_size_15min:]

# Train ARIMA model on 60-second resampled data
arima_60s = ARIMA(train_60s, order=(5, 1, 0))
arima_60s_fit = arima_60s.fit()

# Train ARIMA model on 15-minute data
arima_15min = ARIMA(train_15min, order=(5, 1, 0))
arima_15min_fit = arima_15min.fit()

# Forecasting
arima_60s_forecast = arima_60s_fit.forecast(steps=len(test_60s))
arima_15min_forecast = arima_15min_fit.forecast(steps=len(test_15min))

# Evaluation
arima_60s_mse = mean_squared_error(test_60s, arima_60s_forecast)
arima_15min_mse = mean_squared_error(test_15min, arima_15min_forecast)

print(f'ARIMA 60-second resampled MSE: {arima_60s_mse}')
print(f'ARIMA 15-minute MSE: {arima_15min_mse}')


In [ ]:
# Prophet Modell
import pandas as pd
from prophet import Prophet

# Laden der 60-Sekunden-Daten
df60 = pd.read_csv('cleaned60.csv')
df60['VALUEDATE'] = pd.to_datetime(df60['VALUEDATE'])  # Umwandlung der 'VALUEDATE'-Spalte in Datetime
df60.set_index('VALUEDATE', inplace=True)  # Setzen der 'VALUEDATE'-Spalte als Index

# Laden der 15-Minuten-Daten
df900 = pd.read_csv('cleaned900.csv')
df900['VALUEDATE'] = pd.to_datetime(df900['VALUEDATE'])  # Umwandlung der 'VALUEDATE'-Spalte in Datetime
df900.set_index('VALUEDATE', inplace=True)  # Setzen der 'VALUEDATE'-Spalte als Index

# Fehlende Werte mit dem Mittelwert füllen
df60['avg_value'] = df60.mean(axis=1).fillna(df60.mean(axis=1).mean())
df900['avg_value'] = df900.mean(axis=1).fillna(df900.mean(axis=1).mean())

# Resample 60-Sekunden-Daten auf 15-Minuten-Intervalle
df_60s_resampled = df60['avg_value'].resample('15T').mean().fillna(df60.mean(axis=1).mean())

# Vorbereitung der Daten für Prophet
df_60s_prophet = df_60s_resampled.reset_index().rename(columns={'VALUEDATE': 'ds', 'avg_value': 'y'})
df_15min_prophet = df900.reset_index().rename(columns={'VALUEDATE': 'ds', 'avg_value': 'y'})

# Train-Test Split
train_size = int(len(df_60s_prophet) * 0.8)
train_60s_prophet = df_60s_prophet[:train_size]
test_60s_prophet = df_60s_prophet[train_size:]

train_15min_prophet = df_15min_prophet[:train_size]
test_15min_prophet = df_15min_prophet[train_size:]

# Training des Prophet-Modells auf 60-Sekunden-Daten
model_60s = Prophet()
model_60s.fit(train_60s_prophet)

# Training des Prophet-Modells auf 15-Minuten-Daten
model_15min = Prophet()
model_15min.fit(train_15min_prophet)

# Prognose
future_60s = model_60s.make_future_dataframe(periods=len(test_60s_prophet), freq='15T')
forecast_60s = model_60s.predict(future_60s)

future_15min = model_15min.make_future_dataframe(periods=len(test_15min_prophet), freq='15T')
forecast_15min = model_15min.predict(future_15min)

# Evaluation
from sklearn.metrics import mean_squared_error

test_60s_values = test_60s_prophet['y'].values
forecast_60s_values = forecast_60s.iloc[-len(test_60s_prophet):]['yhat'].values
mse_60s_prophet = mean_squared_error(test_60s_values, forecast_60s_values)

test_15min_values = test_15min_prophet['y'].values
forecast_15min_values = forecast_15min.iloc[-len(test_15min_prophet):]['yhat'].values
mse_15min_prophet = mean_squared_error(test_15min_values, forecast_15min_values)

print(f'Prophet 60-second resampled MSE: {mse_60s_prophet}')
print(f'Prophet 15-minute MSE: {mse_15min_prophet}')
